# **LIBRARIES AND FILE SETUP**

In [ ]:
pip install tsai[extras]

In [ ]:
!git clone https://github.com/fastai/fastai
!pip install -e "fastai[dev]"

In [ ]:
# **************** UNCOMMENT AND RUN THIS CELL IF YOU NEED TO INSTALL/ UPGRADE TSAI ****************
stable = False # Set to True for latest pip version or False for main branch in GitHub
!pip install {"tsai -U" if stable else "git+https://github.com/timeseriesAI/tsai.git"} >> /dev/null

In [ ]:
import tsai

In [ ]:
from tsai.all import *

In [ ]:
import fastai

In [ ]:
from fastai import *
from tsai import *

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [ ]:
import os
os.listdir()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
__all__ = ['df2xy', 'split_xy', 'SlidingWindowSplitter', 'SlidingWindowPanelSplitter', 'prepare_idxs',
           'prepare_sel_vars_and_steps', 'apply_sliding_window', 'df2Xy', 'split_Xy', 'df2np3d',
           'add_missing_value_cols', 'add_missing_timestamps', 'time_encoding', 'forward_gaps', 'backward_gaps',
           'nearest_gaps', 'get_gaps', 'add_delta_timestamp_cols', 'SlidingWindow', 'SlidingWindowPanel',
           'identify_padding']

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
### Set path to the correct folder

path = (r'/content/drive/MyDrive/ColabNotebooks')

In [ ]:
cd /content/drive/MyDrive/ColabNotebooks

# **IMPORT PADDED BATCH FILE**

In [ ]:

#### PADDED DF FROM GOOGLE DRIVE
### -------------------------------------------------------------------------------

### CHEVY BOLT
padded_df = pd.read_csv("/content/drive/MyDrive/ColabNotebooks/Hyundai_Kona_Electric/padded.csv")


In [ ]:
#### ANALYSIS
### -------------------------------------------------------------------------------

padded_df.head()

In [ ]:

### CHECK COL NAMES
### -------------------------------------------------------------------------------

padded_df.columns

In [ ]:

### GENERATE Y FOR PLOT OF DROP OFF LOCATIONS IN CHICARGO
### -------------------------------------------------------------------------------

y = padded_df.groupby("batch_id").last()["dropoff"].values

In [ ]:

### PLOT GRAPH OF DROP OFF LOCATIONS
### -------------------------------------------------------------------------------

plt.figure(figsize=(15,10))
pd.DataFrame(y)[0].value_counts().plot(kind='bar')
plt.title("Distribution of Dropoff batches")

In [ ]:

### CHECK COL NAMES
### -------------------------------------------------------------------------------

padded_df.columns

In [ ]:

### GENERATE DATA_COL VARIABLE
### -------------------------------------------------------------------------------

data_cols = ['range_km', '20_battery', '30_battery', '40_battery', 'battery_Wh',
       'Wh_Km', 'trip_km', 'pickup', 'dropoff', 'start_daymonth',
       'start_dayday', 'start_dayhour', 'end_daymonth', 'end_dayday',
       'end_dayhour', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8',
       'batch_id', 'count']

In [ ]:

### CREATE PADDED DF BY BATCH ID
### -------------------------------------------------------------------------------

padded_df = padded_df.groupby('batch_id')

In [ ]:
padded_df.to_csv("/content/drive/MyDrive/ColabNotebooks/Chevy_Bolt/model_ver1.csv", index=False)

# Experiment 1:20%

In [ ]:
padded_df = pd.read_csv("/content/drive/MyDrive/ColabNotebooks/Chevy_Bolt/model_ver1.csv")

In [ ]:

### ANALYSIS
### -------------------------------------------------------------------------------

padded_df.head()

In [ ]:

### CREATE COMBINED_DF VARIABLE
### -------------------------------------------------------------------------------

combined_df = padded_df.groupby("batch_id")

In [ ]:
combined_df.columns

In [ ]:

### ANALYSIS
### -------------------------------------------------------------------------------

combined_df["batch_id"].value_counts()

In [ ]:

### ANALYSIS
### -------------------------------------------------------------------------------

combined_df["batch_id"].unique()

In [ ]:

### GENERATE VARIABLE CV
### -------------------------------------------------------------------------------

cv  = pd.DataFrame(combined_df.groupby("batch_id").last()["dropoff"])
cv.reset_index(inplace=True)

cv

In [ ]:

### ANALYSIS
### -------------------------------------------------------------------------------

cv["dropoff"].value_counts()

In [ ]:

### SPLIT DATA FOR TRAINING AND TESTING
### -------------------------------------------------------------------------------

X_train, X_val, y_train, y_val = train_test_split(cv[["batch_id"]], cv["dropoff"], test_size=0.15, random_state=123)

In [ ]:

# TESTING AND TRAINING BATCH IDS
### -------------------------------------------------------------------------------

test_batch_ids = X_val["batch_id"].unique()
train_batch_ids = X_train["batch_id"].unique()

len(test_batch_ids), len(train_batch_ids)

In [ ]:

### CREATE TRAINING AND TESTING DATAFRAMES FROM COMMBINED DF BATCH IDS 
### -------------------------------------------------------------------------------

train_df = combined_df[combined_df["batch_id"].isin(train_batch_ids)]
test_df = combined_df[combined_df["batch_id"].isin(test_batch_ids)]

In [ ]:

### ANALYSIS
### -------------------------------------------------------------------------------

train_df.shape, test_df.shape 

In [ ]:

### GENERATE VARIABLES FROM TRAIN AND TEST DF
### -------------------------------------------------------------------------------

train_y = train_df.groupby("batch_id").last()["dropoff"].values
test_y = test_df.groupby("batch_id").last()["dropoff"].values

In [ ]:

### TRAINING - CREATE RES VARIABLE FOR MODEL
### -------------------------------------------------------------------------------

res = df2np3d(train_df, ['batch_id'], data_cols=data_cols )

In [ ]:

### TESTING - CREATE RES VARIABLE FOR MODELLING
### -------------------------------------------------------------------------------

test_res = df2np3d(test_df, ['batch_id'], data_cols=data_cols )

In [ ]:
test_df.shape

In [ ]:
test_df.head()

In [ ]:

### SPLIT TRAINING AND VALIDATION DATA
### -------------------------------------------------------------------------------

splits = get_splits(train_y, valid_size=.15, stratify=True, random_state=23, shuffle=True)
splits

In [ ]:
### 29)
### CREATE IMAGE VARIALBES FOR CNN MODEL
### -------------------------------------------------------------------------------

tfms = [None, Categorize()]
bts = [[TSNormalize(), TSToPlot()], 
       [TSNormalize(), TSToMat(cmap='viridis')],
       [TSNormalize(), TSToGADF(cmap='spring')],
       [TSNormalize(), TSToGASF(cmap='summer')],
       [TSNormalize(), TSToMTF(cmap='autumn')],
       [TSNormalize(), TSToRP(cmap='winter')]]
btns = ['Plot', 'Mat', 'GADF', 'GASF', 'MTF', 'RP']
for i, (bt, btn) in enumerate(zip(bts, btns)):
    dsets = TSDatasets(res, mod_y, tfms=tfms, splits=splits)
    dls = TSDataLoaders.from_dsets(dsets.train,
                                   dsets.valid,
                                   bs=[64, 128],
                                   batch_tfms=bt,
                                   shuffle=False)
    xb, yb = dls.train.one_batch()
    print(f'\n\ntfm: TSTo{btn} - batch shape: {xb.shape}')
    xb[0].show()
    plt.show()

In [ ]:

### CREATE TENSORS FOR MODELLING
### -------------------------------------------------------------------------------

tfms  = [None, [Categorize()]]
dsets = TSDatasets(res, train_y, tfms=tfms, splits=splits, inplace=True, )
dsets

In [ ]:

### TEST VARIABLE FOR MODELLING
### -------------------------------------------------------------------------------

test_ds = dsets.add_test(X= test_res)

In [ ]:

### NEEDED FOR MODELLING TEST RESULTS
### -------------------------------------------------------------------------------

test_dl = TSDataLoaders.from_dsets(test_ds, batch_tfms=[TSStandardize()])

In [ ]:

### NEED FOR BILSTM MODE !
### TRAINING DATA LOADER FOR MODEL
### -------------------------------------------------------------------------------

dls = TSDataLoaders.from_dsets(dsets.train, dsets.valid, batch_tfms=[TSStandardize()], num_workers=0)

In [ ]:

### TEST DATALOADER
### -------------------------------------------------------------------------------

dls.test = dls.add_test(X= test_res)

In [ ]:

### PLOT GRAPHS FROM TRAINING DATA
### -------------------------------------------------------------------------------

dls.show_batch(sharey=True)

# **BiLSTM - TRAINING AND VALIDATION**

In [ ]:
### NEED FOR BILSTM MODE !
### TRAINING DATA LOADER FOR MODEL
### -------------------------------------------------------------------------------

dls = TSDataLoaders.from_dsets(dsets.train, dsets.valid, batch_tfms=[TSStandardize()], num_workers=0)

In [ ]:
### CREATE MODEL VARIABLE FOR BILSTM MODELLING
### -------------------------------------------------------------------------------

model = LSTM(dls.vars, dls.c,n_layers=2, bidirectional=True)
model

In [ ]:
### BILSTM MODELLING - CREATE LEARN VARIABLE FOR FUTURE MODELS
### -------------------------------------------------------------------------------

learn = Learner(dls, model, metrics=accuracy,  loss_func=CrossEntropyLossFlat())
start = time.time()
learn.fit_one_cycle(10, lr_max=1e-3)
print(f"\ntraining time: {time.strftime('%H:%M:%S', time.gmtime(time.time() - start))}")
learn.plot_metrics()

In [ ]:
### BILSTM MODELLING - USING LEARN VARIABLE FROM PREVIOPUS MODEL
### -------------------------------------------------------------------------------

learn.fit_one_cycle(100, lr_max=1e-3)
print(f"\ntraining time: {time.strftime('%H:%M:%S', time.gmtime(time.time() - start))}")
learn.plot_metrics()

# **TEST DATA RESULTS**

In [ ]:
### Import model learner

learn = load_learner('/content/drive/MyDrive/ColabNotebooks/Hyundai_Kona_Electric/pre_trained_learner.pkl')

In [ ]:
op = learn.get_X_preds(test_res, with_decoded=True)

In [ ]:
import ast
y_pred = ast.literal_eval(op[2])

In [ ]:
y_pred[:30]

In [ ]:
y_pred_new = []
for y in y_pred:
    y_pred_new.append(int(y))

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
test_y_new = [int(y) for y in test_y]

In [ ]:
### PRETRAINED MODEL
### Pretrained model 100 - 20% TEST

accuracy_score(test_y_new, y_pred_new)

In [ ]:
### NOT PRETRAINED MODEL
### Not Pretrained model 100 - 20% TEST

accuracy_score(test_y_new, y_pred_new)

# **30% BATTERY CAPACITY PREDICTION**

In [ ]:
### PRE TRAINED LEARNER
### -----------------------------------------------------

learn = load_learner('/content/drive/MyDrive/ColabNotebooks/Chevy_Bolt/not_pretrained_learner.pkl')

In [ ]:
### LOAD TEST DATA

mask_df = pd.read_csv("/content/drive/MyDrive/ColabNotebooks/Chevy_Bolt/mask100.csv") 

In [ ]:
mask_df.columns

In [ ]:
mask_df.shape

In [ ]:
### Drop columns
### -----------------------------------------------------------

to_remove = ["30_battery", "40_battery"]
mask_df.drop(to_remove, axis=1, inplace=True)

In [ ]:
### NEED TO SPLIT DATA FOR TEST PERCENTAGE

cv  = pd.DataFrame(mask_df.groupby("batch_id").last()["dropoff"])
cv.reset_index(inplace=True)

In [ ]:
### NEED TO SPLIT DATA FOR TEST PERCENTAGE

X_train, X_val, y_train, y_val = train_test_split(cv[["batch_id"]], cv["dropoff"], test_size=0.15, random_state=123)

In [ ]:
### NEED TO SPLIT DATA FOR TEST PERCENTAGE

test_batch_ids = X_val["batch_id"].unique()

In [ ]:
### NEED TO SPLIT DATA FOR TEST PERCENTAGE

test_df = mask_df[mask_df["batch_id"].isin(test_batch_ids)]

In [ ]:
test_df.shape

In [ ]:
y = test_df.groupby("batch_id").last()["dropoff"].values

In [ ]:
# low_val = pd.DataFrame(y)[0].value_counts().iloc[-20:].index

low_val = pd.DataFrame(y)[0].value_counts().iloc[0:].index

In [ ]:
### CHECK COLUMNS IN DF

test_df.columns

In [ ]:
test_df["batch_id"].value_counts()

In [ ]:
### MAY NOT NEED ALREADY DONE

data_cols = ['range_km', '20_battery', '30_battery', '40_battery', '70_battery',
       '100_battery', 'battery_Wh', 'Wh_Km', 'trip_km', 'pickup', 'dropoff',
       'start_daymonth', 'start_dayday', 'start_dayhour', 'end_daymonth',
       'end_dayday', 'end_dayhour', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7',
       'b8', 'batch_id', 'count']

In [ ]:
mask_30 = test_df.groupby('batch_id')

In [ ]:
test_30_y = mask_30.groupby("batch_id").last()["dropoff"].values

In [ ]:
mask_30.shape

In [ ]:
test_res = df2np3d(mask_30, ['batch_id'], data_cols=data_cols )

In [ ]:
op = learn.get_X_preds(test_res, with_decoded=True)

In [ ]:
import ast
y_pred = ast.literal_eval(op[2])

In [ ]:
y_pred_new = []
for y in y_pred:
    y_pred_new.append(int(y))

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
test_30_y_new = [int(y) for y in test_30_y]

In [ ]:
len(test_30_y_new), len(y_pred_new)

In [ ]:
### PRE TRAINED LEARNER
### 15% TEST DATASET 

accuracy_score(test_30_y_new, y_pred_new)

In [ ]:
### NOT PRE TRAINED LEARNER
### 15% TEST DATA

accuracy_score(test_30_y_new, y_pred_new)

# **40 % BATTERY CAPACITY PREDICTION**

In [ ]:
### Import model learner

learn = load_learner('/content/drive/MyDrive/ColabNotebooks/Hyundai_Kona_Electric/pre_trained_learner.pkl')

In [ ]:
### LOAD CSV FILE AS DF

mask_40_df = pd.read_csv("/content/drive/MyDrive/ColabNotebooks/Hyundai_Kona_Electric/mask40.csv")

In [ ]:
mask_40_df.shape

In [ ]:
### NEED TO SPLIT DATA FOR TEST PERCENTAGE

cv  = pd.DataFrame(mask_40_df.groupby("batch_id").last()["dropoff"])
cv.reset_index(inplace=True)

In [ ]:
### NEED TO SPLIT DATA FOR TEST PERCENTAGE

X_train, X_val, y_train, y_val = train_test_split(cv[["batch_id"]], cv["dropoff"], test_size=0.15, random_state=123)

In [ ]:
### NEED TO SPLIT DATA FOR TEST PERCENTAGE

test_batch_ids = X_val["batch_id"].unique()

In [ ]:
### NEED TO SPLIT DATA FOR TEST PERCENTAGE

test_df = mask_40_df[mask_40_df["batch_id"].isin(test_batch_ids)]

In [ ]:
### DO

y = test_df.groupby("batch_id").last()["dropoff"].values

In [ ]:
test_df.columns

In [ ]:
### MAY NOT NEED ALREADY DONE

data_cols = ['range_km', '20_battery', '30_battery', '40_battery', 'battery_Wh',
       'Wh_Km', 'trip_km', 'pickup', 'dropoff', 'start_daymonth',
       'start_dayday', 'start_dayhour', 'end_daymonth', 'end_dayday',
       'end_dayhour', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8',
       'batch_id', 'count']

In [ ]:
mask_40 = test_df.groupby('batch_id')

In [ ]:
test_40_y = test_df.groupby("batch_id").last()["dropoff"].values

In [ ]:
test_res = df2np3d(mask_40, ['batch_id'], data_cols=data_cols )

In [ ]:
op = learn.get_X_preds(test_res, with_decoded=True)

In [ ]:
import ast
y_pred = ast.literal_eval(op[2])

In [ ]:
y_pred_new = []
for y in y_pred:
    y_pred_new.append(int(y))

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
test_40_y_new = [int(y) for y in test_40_y]

In [ ]:
len(test_40_y_new), len(y_pred_new)

In [ ]:
### PRE TRAINED LEARNER
### 15% DATASET @ 40% MASKED

accuracy_score(test_40_y_new, y_pred_new)

In [ ]:
### NOT PRE TRAINED LEARNER
### 15% DATASET @ 40% MASKED

accuracy_score(test_40_y_new, y_pred_new)

# **CNN MODELLING**

# **Pre-trained Version - XResNet18**

In [ ]:
### USE FOR MODELLING CNN
### -------------------------------------------------------------------------------

dls = TSDataLoaders.from_dsets(dsets.train, dsets.valid, batch_tfms=[TSNormalize(), TSToMTF(cmap='autumn')], num_workers=0)

In [ ]:
### USE THIS FOR BEST LOOKING PLOTS USING NEXT CELL
### -------------------------------------------------------------------------------

dls = TSDataLoaders.from_dsets(dsets.train, dsets.valid, batch_tfms=[TSNormalize(), TSToPlot()], num_workers=0)

In [ ]:
### RUN TO CREATE BATCH PLOTS
### -------------------------------------------------------------------------------

dls.show_batch()

In [ ]:
### CNN PRE-TRAINED MODEL AND LEARN VARIABLES
### -------------------------------------------------------------------------------

model = create_model(xresnet18, dls=dls, pretrained=True)
learn = Learner(dls, model, metrics=accuracy)

In [ ]:
### CNN PRE-TRAINED MODELLING
### -------------------------------------------------------------------------------

start = time.time()
learn.fit_one_cycle(10, lr_max=1e-3)
print(f"\ntraining time: {time.strftime('%H:%M:%S', time.gmtime(time.time() - start))}")
learn.plot_metrics()

In [ ]:
### CNN PRE-TRAINED MODELLING
### -------------------------------------------------------------------------------

learn.fit_one_cycle(10, lr_max=1e-3)
print(f"\ntraining time: {time.strftime('%H:%M:%S', time.gmtime(time.time() - start))}")
learn.plot_metrics()

In [ ]:
### Export model learner 
### -------------------------------------------------------------------------------

learn.export('/content/drive/MyDrive/ColabNotebooks/Hyundai_Kona_Electric/pre_trained_learner.pkl')

# **NOT Pre-trained Version - XResNet18**

In [ ]:
### MODEL AND LEARN VARIABLES FOR MODEL
### -------------------------------------------------------------------------------

model = create_model(xresnet18, dls=dls, pretrained=False)
learn = Learner(dls, model, metrics=accuracy)

In [ ]:
### MODELLING
### -------------------------------------------------------------------------------

learn.fit_one_cycle(15, lr_max=1e-3)
print(f"\ntraining time: {time.strftime('%H:%M:%S', time.gmtime(time.time() - start))}")
learn.plot_metrics()

In [ ]:
### MODELLING
### -------------------------------------------------------------------------------

learn.fit_one_cycle(20, lr_max=1e-3)
print(f"\ntraining time: {time.strftime('%H:%M:%S', time.gmtime(time.time() - start))}")
learn.plot_metrics()

In [ ]:
### Export model learner 
### -------------------------------------------------------------------------------

learn.export('/content/drive/MyDrive/ColabNotebooks/Hyundai_Kona_Electric/not_pretrained_learner.pkl')